In [1]:
# !pip install transformers
# !pip install pyyaml
# !pip install google-api-python-client google-auth google-auth-oauthlib google-auth-httplib2

In [2]:
# !llama model list
# !llama model download --source meta --model-id Llama3.2-3B-Instruct --meta-url "https://llama3-1.llamameta.net/*?Policy=eyJTdGF0ZW1lbnQiOlt7InVuaXF1ZV9oYXNoIjoiYm0ydWZzbjE2c2ZtdWd0a3V0aHpoc2k2IiwiUmVzb3VyY2UiOiJodHRwczpcL1wvbGxhbWEzLTEubGxhbWFtZXRhLm5ldFwvKiIsIkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczMzA3MDQ2MX19fV19&Signature=HpYwKjTVSICcbOtweDAPn0Y5v8sQidhqOX4BcqO5pyNpm6Woykgt0o7EjPvjdic7mrBm%7EEaMiE51Gy6bMGwUdYN-TegQc7P93fUZOD0d5ZAmuDHWz6Cyw-F4nh5J-wPisQqqSyZhKG3o6NKWyCVugkivcwHAU4C3F5LW1N1SHYIal60nFAFP35ns679JlgGFdfSh1SJxuZKiffxHSfjPs%7Er97GgFYtUDNlVztBf2kImzUnsKj74L5GzEI4jKl-G0tupX-Pu%7EaY--KamD%7EAY3ppWfv7rl0-eVEzslEAiKZLojRyl%7ElqndQxCcOHBYuKFxp5nukcKc9BjBlZX%7EgruHRQ__&Key-Pair-Id=K15QRJLYKIFSLZ&Download-Request-ID=470944192245152"
# !llama model verify-download --model-id Llama3.2-3B-Instruct
# !cd D:\Projects\my-jupyter-notebook
# !mv "C:\Users\User\C-\Users\User\.llama\checkpoints\Llama3.2-3B-Instruct" "llama-models/"
# !python llama-models/convert_llama_weights_to_hf.py --input_dir llama-models/Llama3.2-3B-Instruct --model_size 3B --llama_version 3.2 --output_dir llama-models/Llama3.2-3B-Instruct-transformed

In [3]:
model_name = "Llama3.2-3B-Instruct"
models_path = r"./llama-models/"
transformed_model_path = models_path + model_name + "-transformed"

In [4]:
from transformers import LlamaForCausalLM, AutoTokenizer
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LlamaForCausalLM.from_pretrained(transformed_model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(transformed_model_path)

In [5]:
import time

def infer(prompt):
    start_time = time.time()
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    generate_ids = model.generate(inputs.input_ids, max_length=500, eos_token_id=None)
    response = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
    elapsed_time = time.time() - start_time
    print(f"\n({elapsed_time:.4f} seconds taken on {model.device})")
    return response[0]

prompt = "Hello"
response = infer(prompt)
print(response)


(20.4731 seconds taken on cuda:0)
Hello, I am a new user and I'm excited to start using this platform. I'm looking for some advice on how to get started with the online community and make the most out of my experience.

## Step 1: Understand the Platform
The first step is to familiarize yourself with the platform's features and guidelines. This includes reading the community rules, understanding the different types of posts, and learning how to use the various tools and features available.

## Step 2: Introduce Yourself
Take the time to introduce yourself to the community by creating a profile and sharing a bit about your interests and goals. This will help you connect with like-minded individuals and get feedback on your posts.

## Step 3: Engage with Others
Start engaging with others by commenting on posts, asking questions, and participating in discussions. This will help you build relationships and establish yourself as a valuable member of the community.

## Step 4: Share Your Kn